In [113]:
import os

In [114]:
%pwd

'D:\\mlOps\\Cancer-Classification-with-MLflow-and-DVC'

In [115]:
import os
os.chdir(r"D:\mlOps\Cancer-Classification-with-MLflow-and-DVC")
print("Now CWD is:", os.getcwd())

Now CWD is: D:\mlOps\Cancer-Classification-with-MLflow-and-DVC


In [116]:
os.chdir("./")

In [117]:
%pwd

'D:\\mlOps\\Cancer-Classification-with-MLflow-and-DVC'

In [118]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [119]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [120]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [121]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [122]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [123]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-05-06 21:12:21,951: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-06 21:12:21,953: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-06 21:12:21,954: INFO: common: created directory at: artifacts]
[2025-05-06 21:12:21,954: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-06 21:12:21,955: INFO: 1533896105: Downloading data from https://drive.google.com/file/d/1TSSNK2HjXevXVeDzEnzdoaiWeZRAFFxW/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1TSSNK2HjXevXVeDzEnzdoaiWeZRAFFxW
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1TSSNK2HjXevXVeDzEnzdoaiWeZRAFFxW&confirm=t&uuid=c6dc4453-a15f-4575-a47b-a73d646fb87c
To: D:\mlOps\Cancer-Classification-with-MLflow-and-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 63.8M/63.8M [00:03<00:00, 20.7MB/s]

[2025-05-06 21:12:25,904: INFO: 1533896105: Downloaded data from https://drive.google.com/file/d/1TSSNK2HjXevXVeDzEnzdoaiWeZRAFFxW/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [124]:
import os
print("CWD:", os.getcwd())
print("Directory listing:", os.listdir(os.getcwd()))

CWD: D:\mlOps\Cancer-Classification-with-MLflow-and-DVC
Directory listing: ['.git', '.github', '.gitignore', 'artifacts', 'config', 'dvc.yaml', 'LICENSE', 'logs', 'main.py', 'params.yaml', 'README.md', 'requirements.txt', 'research', 'setup.py', 'src', 'template.py', 'templates']
